In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import pandas as pd
import time
import sqlite3

In [2]:
year = "2023"
s = Service('/chromedriver/chromedriver.exe')
driver = webdriver.Chrome(service = s) 

In [3]:
player_stats_url = "https://www.basketball-reference.com/leagues/NBA_{}_per_game.html"

url = player_stats_url.format(year)
driver.get(url)
html = driver.page_source #grab the data
with open("player/{}.html".format(year),"w+",encoding="utf-8") as f:
    f.write(html)

In [4]:
dfs2 = []

with open("player/{}.html".format(year),encoding="utf-8") as f:
    page = f.read()
soup = BeautifulSoup(page,"html.parser")
tHeads = soup.findAll('tr', class_="thead")
for tHead in tHeads:
    tHead.decompose()
player_table = soup.find(id = "per_game_stats")
player = pd.read_html(str(player_table))[0]
player["Year"] = year
dfs2.append(player)
players = pd.concat(dfs2)
players

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year
0,1,Precious Achiuwa,C,23,TOR,54,11,20.6,3.5,7.3,...,1.8,4.0,5.8,0.9,0.5,0.6,1.1,1.9,9.1,2023
1,2,Steven Adams,C,29,MEM,42,42,27.0,3.7,6.3,...,5.1,6.5,11.5,2.3,0.9,1.1,1.9,2.3,8.6,2023
2,3,Bam Adebayo,C,25,MIA,74,74,35.0,8.1,15.0,...,2.5,6.8,9.3,3.2,1.2,0.8,2.5,2.8,20.6,2023
3,4,Ochai Agbaji,SG,22,UTA,57,20,20.1,2.6,6.1,...,0.7,1.3,2.1,1.0,0.2,0.3,0.6,1.7,7.6,2023
4,5,Santi Aldama,PF,22,MEM,77,20,21.8,3.2,6.8,...,1.1,3.7,4.8,1.3,0.6,0.6,0.8,1.9,9.0,2023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
670,531,Thaddeus Young,PF,34,TOR,54,9,14.7,2.0,3.7,...,1.3,1.8,3.1,1.4,1.0,0.1,0.8,1.6,4.4,2023
671,532,Trae Young,PG,24,ATL,73,73,34.8,8.2,19.0,...,0.8,2.2,3.0,10.2,1.1,0.1,4.1,1.4,26.2,2023
672,533,Omer Yurtseven,C,24,MIA,8,0,7.4,1.4,2.3,...,0.8,1.3,2.0,0.3,0.1,0.1,0.4,1.5,3.3,2023
673,534,Cody Zeller,C,30,MIA,14,2,14.6,2.5,4.1,...,1.7,2.6,4.3,0.6,0.2,0.3,1.0,2.2,6.6,2023


In [6]:
playerscon = sqlite3.connect('players.sqlite')

In [7]:
cursor = playerscon.cursor()
cursor.execute("DELETE FROM PLAYERS WHERE year = ?", (2023,))
# Commit the changes
playerscon.commit()
# Close the cursor and connection
cursor.close()

In [8]:
players.to_sql("PLAYERS", con = playerscon,if_exists = 'append')

675

In [9]:
team_stats_url = "https://www.basketball-reference.com/leagues/NBA_{}_standings.html"

url = team_stats_url.format(year)
driver.get(url)
html = driver.page_source #grab the data
with open("team/{}.html".format(year),"w+",encoding="utf-8") as f:
    f.write(html)

In [10]:
dfs3 = []


with open("team/{}.html".format(year),encoding="utf-8") as f:
    page = f.read()
soup = BeautifulSoup(page,"html.parser")
if(soup.find('tr', class_="thead")!=None):
     soup.find('tr', class_="thead").decompose()
team_table = soup.find(id = "divs_standings_E")
team = pd.read_html(str(team_table))[0]
team["Year"] = year
team["Team"] = team["Eastern Conference"]
del team["Eastern Conference"]
dfs3.append(team)

soup = BeautifulSoup(page,"html.parser")
if(soup.find('tr', class_="thead")!=None):

    soup.find('tr', class_="thead").decompose()

team_table = soup.find(id = "divs_standings_W")
team = pd.read_html(str(team_table))[0]
team["Year"] = year
team["Team"] = team["Western Conference"]
del team["Western Conference"]
dfs3.append(team)
teams = pd.concat(dfs3)

In [11]:
teamscon = sqlite3.connect('teams.sqlite')

In [12]:
cursor = teamscon.cursor()
cursor.execute("DELETE FROM TEAMS WHERE year = ?", (2023,))
# Commit the changes
teamscon.commit()
# Close the cursor and connection
cursor.close()

In [13]:
teams.to_sql("TEAMS", con = teamscon,if_exists = 'append')

35